In [2]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr

# plot
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.ticker as mticker
from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cmocean as ccm

import matplotlib.style as style
style.use('tableau-colorblind10')

CB_colors = {
    'blue':    '#377eb8', 
    'orange':  '#ff7f00',
    'green':   '#4daf4a',
    'pink':    '#f781bf',
    'brown':   '#a65628',
    'purple':  '#984ea3',
    'gray':    '#999999',
    'red':     '#e41a1c',
    'yellow':  '#dede00'
} 

In [3]:
from dask.distributed import Client
c = Client()
# c = Client(n_workers=6, threads_per_worker=1, memory_limit="8 GiB",local_directory="/gpfsscratch/rech/cli/uor98hu/dask_tmp/")
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36621 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:36621/status,
Dashboard: http://127.0.0.1:36621/status,Workers: 5
Total threads: 10,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38291,Workers: 5
Dashboard: http://127.0.0.1:36621/status,Total threads: 10
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:39099,Total threads: 2
Dashboard: http://127.0.0.1:39063/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:42859,


# Load data

In [4]:
# loading data
chunk_size = {"x":360,"y":500} 
dirww='/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/cumsum/'

y1='2005'
y2='2018'

dsGAI = xr.open_dataset(dirww +'eORCA025.L75-IMHOTEP.GAI_y'+y1+'_'+y2+'.1y_components_0_bottom_masked.nc', chunks=chunk_size)
dsAI = xr.open_dataset(dirww +'eORCA025.L75-IMHOTEP.AI_y'+y1+'_'+y2+'.1y_components_0_bottom_masked.nc', chunks=chunk_size)
dsS = xr.open_dataset(dirww +'eORCA025.L75-IMHOTEP.S_y'+y1+'_'+y2+'.1y_components_0_bottom_masked.nc', chunks=chunk_size)

stericGAI = dsGAI.steric
halostericGAI = dsGAI.halosteric
thermostericGAI = dsGAI.thermosteric

stericAI = dsAI.steric
halostericAI = dsAI.halosteric
thermostericAI = dsAI.thermosteric

stericS = dsS.steric
halostericS= dsS.halosteric
thermostericS = dsS.thermosteric


In [5]:
# compute the sensitivy differences

# GAI - S (Gr+Riv)
stericGAI_S = stericGAI -  stericS
halostericGAI_S = halostericGAI - halostericS
thermostericGAI_S = thermostericGAI -  thermostericS

# GAI- AI (GREENLAND)
stericGAI_AI = stericGAI -  stericAI
halostericGAI_AI = halostericGAI - halostericAI
thermostericGAI_AI = thermostericGAI -  thermostericAI

# AI - S (RIVERS)
stericAI_S = stericAI - stericS
halostericAI_S = halostericAI - halostericS
thermostericAI_S = thermostericAI - thermostericS

# Saving trends

## GAI

In [4]:
dmax=75
maskBGdims = np.shape(stericS) # (14, 75, 91, 123)


xrtrhaloGAI = np.zeros((dmax,maskBGdims[2],maskBGdims[3])) # z,y,x
xrtrthermoGAI = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrstericGAI = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))

for prof in np.arange(dmax):

    xr1,xrorigins1,years1,xrtrstd_GAI_AIhalo  = slx.trpolyfit(halostericGAI.isel(deptht=prof),'yr')
    xr2,xrorigins2,years2,xrtrstd_GAI_AIthermo= slx.trpolyfit(thermostericGAI.isel(deptht=prof),'yr')
    xr3,xrorigins3,years3,xrtrstd_AI_AIsteric = slx.trpolyfit(stericGAI.isel(deptht=prof),'yr')
    
    xrtrhaloGAI[prof,:,:] = xr1
    xrtrthermoGAI[prof,:,:] = xr2
    xrtrstericGAI[prof,:,:] = xr3
    
    
ds = xr.Dataset(
    data_vars=dict(
        xrtrstericGAI=(["deptht","y", "x"], xrtrstericGAI),
        xrtrhaloGAI=(["deptht","y", "x"], xrtrhaloGAI),
        xrtrthermoGAI=(["deptht","y", "x"], xrtrthermoGAI),
        ),

    coords=dict(
        deptht=(["deptht"], stericGAI.deptht[:dmax].values),
        ),

    attrs=dict(description="trend in the BGRmaskobs over "+y1+' '+y2+" for GAI "),

)

ds.to_netcdf(path=dirww+'eORCA025.L75-IMHOTEP.GAI_y'+y1+'_'+y2+'.1y_componentstrends_0_bottom_masked.nc', mode='w')


## GAI - S

In [5]:
dmax=75
maskBGdims = np.shape(stericS)

xrtrhaloGAI_S = np.zeros((dmax,maskBGdims[2],maskBGdims[3])) # z,y,x
xrtrthermoGAI_S = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrstericGAI_S = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))

for prof in np.arange(dmax):

    xr1,xrorigins1,years1,xrtrstdhalo  = slx.trpolyfit(halostericGAI_S.isel(deptht=prof),'yr')
    xr2,xrorigins2,years2,xrtrstdthermo= slx.trpolyfit(thermostericGAI_S.isel(deptht=prof),'yr')
    xr3,xrorigins3,years3,xrtrstdteric = slx.trpolyfit(stericGAI_S.isel(deptht=prof),'yr')
    
    xrtrhaloGAI_S[prof,:,:] = xr1
    xrtrthermoGAI_S[prof,:,:] = xr2
    xrtrstericGAI_S[prof,:,:] = xr3
    
    
ds = xr.Dataset(
    data_vars=dict(
        xrtrstericGAI_S=(["deptht","y", "x"], xrtrstericGAI_S),
        xrtrhaloGAI_S=(["deptht","y", "x"], xrtrhaloGAI_S),
        xrtrthermoGAI_S=(["deptht","y", "x"], xrtrthermoGAI_S),
        ),

    coords=dict(
        deptht=(["deptht"], stericGAI.deptht[:dmax].values),
        ),

    attrs=dict(description="trend in the BGRmaskobs over "+y1+' '+y2+" for GAI - S "),

)

ds.to_netcdf(path=dirww+'eORCA025.L75-IMHOTEP.GAI_S_y'+y1+'_'+y2+'.1y_componentstrends_0_bottom_masked.nc', mode='w')


## GAI - AI

In [7]:
dmax=75
maskBGdims = np.shape(stericS)

xrtrhaloGAI_AI = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrthermoGAI_AI = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrstericGAI_AI = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))

for prof in np.arange(dmax):

    xr1,xrorigins1,years1,xrtrstd_GAI_AIhalo  = slx.trpolyfit(halostericGAI_AI.isel(deptht=prof),'yr')
    xr2,xrorigins2,years2,xrtrstd_GAI_AIthermo= slx.trpolyfit(thermostericGAI_AI.isel(deptht=prof),'yr')
    xr3,xrorigins3,years3,xrtrstd_AI_AIsteric = slx.trpolyfit(stericGAI_AI.isel(deptht=prof),'yr')
    
    xrtrhaloGAI_AI[prof,:,:] = xr1
    xrtrthermoGAI_AI[prof,:,:] = xr2
    xrtrstericGAI_AI[prof,:,:] = xr3
    
    
ds = xr.Dataset(
    data_vars=dict(
        xrtrstericGAI_AI=(["deptht","y", "x"], xrtrstericGAI_AI),
        xrtrhaloGAI_AI=(["deptht","y", "x"], xrtrhaloGAI_AI),
        xrtrthermoGAI_AI=(["deptht","y", "x"], xrtrthermoGAI_AI),
        ),

    coords=dict(
        deptht=(["deptht"], stericGAI.deptht[:dmax].values),
        ),

    attrs=dict(description="trend in the BGRmaskobs over "+y1+' '+y2+" for GAI -AI"),

)

ds.to_netcdf(path=dirww+'eORCA025.L75-IMHOTEP.GAI_AI_y'+y1+'_'+y2+'.1y_componentstrends_0_bottom_masked.nc', mode='w')

CPU times: user 1min 47s, sys: 1min 59s, total: 3min 46s
Wall time: 46.8 s


## AI - S

In [7]:
dmax=75
maskBGdims = np.shape(stericS)

xrtrhaloAI_S   = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrthermoAI_S = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))
xrtrstericAI_S = np.zeros((dmax,maskBGdims[2],maskBGdims[3]))

for prof in np.arange(dmax):

    xr1,xrorigins1,years1,xrtrstdhalo  = slx.trpolyfit(halostericAI_S.isel(deptht=prof),'yr')
    xr2,xrorigins2,years2,xrtrstdthermo= slx.trpolyfit(thermostericAI_S.isel(deptht=prof),'yr')
    xr3,xrorigins3,years3,xrtrstdsteric = slx.trpolyfit(stericAI_S.isel(deptht=prof),'yr')
    
    xrtrhaloAI_S[prof,:,:] = xr1
    xrtrthermoAI_S[prof,:,:] = xr2
    xrtrstericAI_S[prof,:,:] = xr3
    
    
ds = xr.Dataset(
    data_vars=dict(
        xrtrstericAI_S=(["deptht","y", "x"], xrtrstericAI_S),
        xrtrhaloAI_S  =(["deptht","y", "x"], xrtrhaloAI_S),
        xrtrthermoAI_S=(["deptht","y", "x"], xrtrthermoAI_S),
        ),

    coords=dict(
        deptht=(["deptht"], stericGAI.deptht[:dmax].values),
        ),

    attrs=dict(description="trend in the BGRmaskobs over "+y1+' '+y2+" for AI-S"),

)

ds.to_netcdf(path=dirww+'eORCA025.L75-IMHOTEP.AI_S_y'+y1+'_'+y2+'.1y_componentstrends_0_bottom_masked.nc', mode='w')

# PLOTTING FIGURE

## loading data

In [6]:
y1='2005'
y2='2018'

BGRmask = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc').BGRmask.squeeze()

diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh = xr.open_dataset(diri+'mesh_hgr.nc').squeeze()
e1t = mesh.e1t.where(BGRmask,drop=True)
e2t = mesh.e2t.where(BGRmask,drop=True)
cellArea=e1t*e2t
AreaTot = cellArea.sum(("y","x")).compute()
weight = cellArea/AreaTot

In [7]:
dirww='/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/cumsum/' 
GAI_Strends = xr.open_dataset(dirww +'eORCA025.L75-IMHOTEP.GAI_S_y2005_2018.1y_componentstrends_0_bottom_masked.nc') *1000
GAI_AItrends = xr.open_dataset(dirww+'eORCA025.L75-IMHOTEP.GAI_AI_y2005_2018.1y_componentstrends_0_bottom_masked.nc') *1000
AI_Strends = xr.open_dataset(dirww  +'eORCA025.L75-IMHOTEP.AI_S_y2005_2018.1y_componentstrends_0_bottom_masked.nc')    *1000

GAI_Str_st= GAI_Strends.xrtrstericGAI_S
GAI_Str_ha= GAI_Strends.xrtrhaloGAI_S
GAI_Str_th= GAI_Strends.xrtrthermoGAI_S

GAI_AItr_st= GAI_AItrends.xrtrstericGAI_AI
GAI_AItr_ha= GAI_AItrends.xrtrhaloGAI_AI
GAI_AItr_th= GAI_AItrends.xrtrthermoGAI_AI

AI_Str_st= AI_Strends.xrtrstericAI_S
AI_Str_ha= AI_Strends.xrtrhaloAI_S
AI_Str_th= AI_Strends.xrtrthermoAI_S

In [8]:
# weighted mean in the box
meanGAI_Str_st = (GAI_Str_st*weight).sum(('x','y'))
meanGAI_Str_ha = (GAI_Str_ha*weight).sum(('x','y'))
meanGAI_Str_th = (GAI_Str_th*weight).sum(('x','y'))


meanGAI_AItr_st = (GAI_AItr_st*weight).sum(('x','y'))
meanGAI_AItr_ha = (GAI_AItr_ha*weight).sum(('x','y'))
meanGAI_AItr_th = (GAI_AItr_th*weight).sum(('x','y'))

meanAI_Str_st = (AI_Str_st*weight).sum(('x','y'))
meanAI_Str_ha = (AI_Str_ha*weight).sum(('x','y'))
meanAI_Str_th = (AI_Str_th*weight).sum(('x','y'))

# Plot figure

In [17]:
dmax = 59
fig, [ax0,ax1,ax2] = plt.subplots(1,3, figsize=([15,8]),facecolor='none', dpi=72*2,sharey=True, gridspec_kw = {'wspace':0.09})

## --- GR + RIV

meanGAI_Str_st.isel(deptht=np.arange(dmax)).plot(ax=ax0, color='k',y='deptht',yincrease=False, linewidth=2)*1000
meanGAI_Str_ha.isel(deptht=np.arange(dmax)).plot(ax=ax0, color=CB_colors['blue'],y='deptht',yincrease=False,linestyle='dashed', linewidth=2)*1000
meanGAI_Str_th.isel(deptht=np.arange(dmax)).plot(ax=ax0, color=CB_colors['red'],y='deptht',yincrease=False,linestyle='dotted', linewidth=2)*1000
ax0.plot([meanGAI_Str_th.min().values,2.1],[meanGAI_AItr_st.deptht[34],meanGAI_AItr_st.deptht[34]], '--', color=CB_colors['gray'])
ax0.set_ylim(2800,-5)
ax0.set_xlim(-0.1,2.1)

ax0.legend(['Steric','Halosteric','Thermosteric'], fontsize=16,loc="lower center")
ax0.set_xlabel(r'Anomaly [$mm~yr^{-1}$]', fontsize=14)
ax0.tick_params(axis='both', labelsize=14)
ax0.set_ylabel(r'Depth $[m]$', fontsize=14)
ax0.set_title('Gr+Riv',fontsize=14)
ax0.grid()
ax0.axvline(0, color='black')


## --- GREENLAND

meanGAI_AItr_st.isel(deptht=np.arange(dmax)).plot(ax=ax1, color='k',y='deptht',yincrease=False, linewidth=2)*1000
meanGAI_AItr_ha.isel(deptht=np.arange(dmax)).plot(ax=ax1, color=CB_colors['blue'],y='deptht',yincrease=False,linestyle='dashed', linewidth=2)*1000
meanGAI_AItr_th.isel(deptht=np.arange(dmax)).plot(ax=ax1, color=CB_colors['red'],y='deptht',yincrease=False,linestyle='dotted', linewidth=2)*1000
ax1.plot([meanGAI_AItr_th.min().values,meanGAI_AItr_ha.max().values],[meanGAI_AItr_st.deptht[34],meanGAI_AItr_st.deptht[34]], '--', color=CB_colors['gray'])
ax1.set_ylim(2800,-5)
ax1.set_xlim(-0.1,2.1)

ax1.legend(['Steric','Halosteric','Thermosteric'], fontsize=16,loc="lower center")
ax1.set_xlabel(r'Anomaly [$mm~yr^{-1}$]', fontsize=14)
ax1.tick_params(axis='both', labelsize=14)
ax1.set_title('GREENLAND',fontsize=14)
ax1.set_ylabel('')
ax1.axvline(0, color='black')

ax1.grid()

## --- RIV

meanAI_Str_st.isel(deptht=np.arange(dmax)).plot(ax=ax2, color='k',y='deptht',yincrease=False, linewidth=2)*1000
meanAI_Str_ha.isel(deptht=np.arange(dmax)).plot(ax=ax2, color=CB_colors['blue'],y='deptht',yincrease=False,linestyle='dashed', linewidth=2)*1000
meanAI_Str_th.isel(deptht=np.arange(dmax)).plot(ax=ax2, color=CB_colors['red'],y='deptht',yincrease=False,linestyle='dotted', linewidth=2)*1000
ax2.plot([meanAI_Str_ha.min().values,meanAI_Str_th.max().values],[meanAI_Str_st.deptht[34],meanAI_Str_st.deptht[34]], '--', color=CB_colors['gray'])
ax2.set_ylim(2800,-5)
ax2.set_xlim(-2.1,0.1)

ax2.legend(['Steric','Halosteric','Thermosteric'], fontsize=16,loc="lower center")
ax2.set_xlabel(r'Anomaly [$mm~yr^{-1}$]', fontsize=14)
ax2.tick_params(axis='both', labelsize=14)
ax2.set_title('RIVERS',fontsize=14)
ax2.set_ylabel('')
ax2.grid()
ax2.axvline(0, color='black')


ax0.text(0, 1.02, 'a)', fontsize=14, horizontalalignment='center', verticalalignment='center', transform=ax0.transAxes)
ax1.text(0, 1.02, 'b)', fontsize=14, horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
ax2.text(0, 1.02, 'c)', fontsize=14, horizontalalignment='center', verticalalignment='center', transform=ax2.transAxes)

diro = "/gpfsscratch/rech/cli/uor98hu/PLOTS/" 
namo = "fig_4_vertical_profile_4paper.png"
print(namo)
# plt.show()
fig.savefig(diro+namo, facecolor=fig.get_facecolor(), pad_inches=0.1, bbox_inches='tight') #,edgecolor='none', bbox_inches='tight'
plt.close()

fig_4_vertical_profile_4paper.png
